In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pulp as plp
import networkx as nx
from ALB_instance_tools import *


In [2]:
def make_scenario_tree(n_takts, entry_probabilities):
    """
    Creates a scenario tree for the given number of takts, instance and entry probabilities.
    """
    # Create a directed graph
    G = nx.DiGraph()
    # Add the root node
    G.add_node('R', stage=0, scenario=0)
    #Create a list of final sequences
    final_sequences = {}
    def add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability=1, parent=0, sequence = [], current_stage=0, counter=[0]):
        if current_stage == n_takts:
            final_sequences[counter[0]] = {'sequence':sequence, 'probability': probability}
            counter[0] += 1
            return
        else:
            for model, prob in entry_probabilities.items():
                new_sequence = sequence.copy()
                new_sequence.append(model)
                node_name = str(parent)+ str(model) 
                graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                graph.add_edge(parent, node_name, probability = probability)
                add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1)
    add_nodes(n_takts, entry_probabilities, G, final_sequences, parent='R')
    return G, final_sequences
NO_takts = 4
scenario_tree_graph, final_sequences = make_scenario_tree(NO_takts, {'A':0.75, 'B':0.25})
final_sequences

{0: {'sequence': ['A', 'A', 'A', 'A'], 'probability': 0.31640625},
 1: {'sequence': ['A', 'A', 'A', 'B'], 'probability': 0.10546875},
 2: {'sequence': ['A', 'A', 'B', 'A'], 'probability': 0.10546875},
 3: {'sequence': ['A', 'A', 'B', 'B'], 'probability': 0.03515625},
 4: {'sequence': ['A', 'B', 'A', 'A'], 'probability': 0.10546875},
 5: {'sequence': ['A', 'B', 'A', 'B'], 'probability': 0.03515625},
 6: {'sequence': ['A', 'B', 'B', 'A'], 'probability': 0.03515625},
 7: {'sequence': ['A', 'B', 'B', 'B'], 'probability': 0.01171875},
 8: {'sequence': ['B', 'A', 'A', 'A'], 'probability': 0.10546875},
 9: {'sequence': ['B', 'A', 'A', 'B'], 'probability': 0.03515625},
 10: {'sequence': ['B', 'A', 'B', 'A'], 'probability': 0.03515625},
 11: {'sequence': ['B', 'A', 'B', 'B'], 'probability': 0.01171875},
 12: {'sequence': ['B', 'B', 'A', 'A'], 'probability': 0.03515625},
 13: {'sequence': ['B', 'B', 'A', 'B'], 'probability': 0.01171875},
 14: {'sequence': ['B', 'B', 'B', 'A'], 'probability': 0.0

In [3]:
# instance_dicts = [
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_2.alb",'name':'B', 'probability':0.25}
# ]

instance_dicts = [
   {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",'name':'A', 'probability':0.75},
   {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",'name':'B', 'probability':0.25}
]
test_instances = create_instance_pair_stochastic(instance_dicts)


NO_EQUIPMENT = 4
#TODO : Change to allow for different number of tasks for each model

all_tasks = get_task_union(test_instances, 'A', 'B')
NO_TASKS = len(all_tasks)
seed = 1
NO_WORKERS =4
NO_STATIONS = 3
WORKER_COST = 500
TAKT_TIME = 300

In [4]:
all_tasks

{'1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9'}

In [5]:

c_se, r_oe = generate_equipment(NO_EQUIPMENT, NO_STATIONS, all_tasks, seed = 550)
equipment_instance = {1: {'equipment_prices': c_se, 'equipment_matrix': r_oe}}

In [6]:
r_oe

array([[ True,  True,  True,  True],
       [ True,  True, False,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False,  True],
       [ True,  True, False,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [7]:
c_se[1,2]

265

In [8]:
def check_scenarios(prod_sequence1,prod_sequence2,t):
    '''compares two production sequences up to time t and returns true if they are the same'''
    if prod_sequence1[:t+1] == prod_sequence2[:t+1]:
        return True
    else:
        return False


In [9]:
prob = 0
for sequence in final_sequences.items():
    prob +=sequence[1]['probability']
    print(sequence)
print(prob)

(0, {'sequence': ['A', 'A', 'A', 'A'], 'probability': 0.31640625})
(1, {'sequence': ['A', 'A', 'A', 'B'], 'probability': 0.10546875})
(2, {'sequence': ['A', 'A', 'B', 'A'], 'probability': 0.10546875})
(3, {'sequence': ['A', 'A', 'B', 'B'], 'probability': 0.03515625})
(4, {'sequence': ['A', 'B', 'A', 'A'], 'probability': 0.10546875})
(5, {'sequence': ['A', 'B', 'A', 'B'], 'probability': 0.03515625})
(6, {'sequence': ['A', 'B', 'B', 'A'], 'probability': 0.03515625})
(7, {'sequence': ['A', 'B', 'B', 'B'], 'probability': 0.01171875})
(8, {'sequence': ['B', 'A', 'A', 'A'], 'probability': 0.10546875})
(9, {'sequence': ['B', 'A', 'A', 'B'], 'probability': 0.03515625})
(10, {'sequence': ['B', 'A', 'B', 'A'], 'probability': 0.03515625})
(11, {'sequence': ['B', 'A', 'B', 'B'], 'probability': 0.01171875})
(12, {'sequence': ['B', 'B', 'A', 'A'], 'probability': 0.03515625})
(13, {'sequence': ['B', 'B', 'A', 'B'], 'probability': 0.01171875})
(14, {'sequence': ['B', 'B', 'B', 'A'], 'probability': 0.0

In [10]:
def add_non_aticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, num_stations):
    '''adds the non participation constraint for two scenarios w and w_prime'''
    #We go backwards in time, look for first time t where the sequences are the same
    for t in reversed(range(sequence_length)):
        if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], t):
            for j in range(t+1):
                model = prod_sequences[w]['sequence'][j]
                max_station = min(t-j+1, num_stations)    
                for s in range(max_station):
                    for o in range(model_instance[model]['num_tasks']):
                        prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                                f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')
            return


In [11]:
r_oe.shape

(20, 4)

In [12]:


def stochastic_problem_linear_labour(model_instance, equipment_instance,NO_TASKS, NO_WORKERS, NO_STATIONS,takt_time, sequence_length, prod_sequences, worker_cost =100):
    print('Writing problem')
    print('Number of tasks:', NO_TASKS, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (final_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_wsoj = plp.LpVariable.dicts('x_wsoj', (final_sequences.keys(), stations, range(NO_TASKS), range(sequence_length) ), lowBound=0, cat='Binary')
    #Defining LP problem
    prob = plp.LpProblem("Stochastic_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [final_sequences[w]['probability']/sequence_length * worker_cost* l * b_wtsl[w][t][s][l]
                         for w in prod_sequences.keys()
                           for t in range(0,sequence_length) 
                           for s in stations for l in workers]),
                "Total cost")
    #Constraints
    #constraints work over all scenarios
    # for w in final_sequences.keys():
        #Constraint 1 -- can only assign l number of workers to a station for a given scenario and stage
    for w in final_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
        #Constraint 2 all tasks must be assigned to a station
    for w in final_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
            #but this is more general
            for o in range(model_instance[model]['num_tasks']): 
                prob += (plp.lpSum([x_wsoj[w][s][o][j] for s in stations]) == 1, f'x_wsoj_{w}_s_{o}_{j}')
        #Constraint 3 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in final_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_wsoj[w][s][int(o)-1][j] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 4 -- tasks can only be assigned to a station with the correct equipment
    for w in final_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for s in stations:
                for o in range(model_instance[model]['num_tasks']):
                    prob += x_wsoj[w][s][o][j] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_wsoj_{w}_{s}_{o}_{j}'
        #Constraint 5 -- precedence constraints
    for w in final_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for (pred, suc) in model_instance[model]['precedence_relations']:
                prob += (plp.lpSum([ (s+1)  * x_wsoj[w][s][int(pred)-1][j] for s in stations])
                         <=  
                         plp.lpSum([ (s+1)  * x_wsoj[w][s][int(suc)-1][j] for s in stations]), 
                         f'task{pred} before task{suc} for model{model}, item {j} seq {w}' )
        #Constraint 6 -- non-anticipativity constraints
    for w in final_sequences.keys():
        for w_prime in prod_sequences.keys():
            if w_prime > w:
                add_non_aticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, NO_STATIONS)
                # for j in reversed(range(sequence_length)):
                #     if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], j):
                #         model = prod_sequences[w]['sequence'][j]
                #         print('w', w, 'w_prime', w_prime)
                #         print('j', j, 'max_station', NO_STATIONS-j, 'model', model)
                #         for s in range(NO_STATIONS - j ):
                #             for o in range(model_instance[model]['num_tasks']):
                #                 prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                #                         f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')

                
    return prob
prob = stochastic_problem_linear_labour(test_instances, equipment_instance[1],NO_TASKS, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_takts, final_sequences, worker_cost=WORKER_COST)                                


Writing problem
Number of tasks: 20 Number of workers: 4 
 Number of stations: 3 Takt time: 300 Sequence length: 4 worker_cost: 500


In [13]:
prob

Stochastic_problem:
MINIMIZE
39.55078125*b_wtsl_0_0_0_1 + 79.1015625*b_wtsl_0_0_0_2 + 118.65234375*b_wtsl_0_0_0_3 + 158.203125*b_wtsl_0_0_0_4 + 39.55078125*b_wtsl_0_0_1_1 + 79.1015625*b_wtsl_0_0_1_2 + 118.65234375*b_wtsl_0_0_1_3 + 158.203125*b_wtsl_0_0_1_4 + 39.55078125*b_wtsl_0_0_2_1 + 79.1015625*b_wtsl_0_0_2_2 + 118.65234375*b_wtsl_0_0_2_3 + 158.203125*b_wtsl_0_0_2_4 + 39.55078125*b_wtsl_0_1_0_1 + 79.1015625*b_wtsl_0_1_0_2 + 118.65234375*b_wtsl_0_1_0_3 + 158.203125*b_wtsl_0_1_0_4 + 39.55078125*b_wtsl_0_1_1_1 + 79.1015625*b_wtsl_0_1_1_2 + 118.65234375*b_wtsl_0_1_1_3 + 158.203125*b_wtsl_0_1_1_4 + 39.55078125*b_wtsl_0_1_2_1 + 79.1015625*b_wtsl_0_1_2_2 + 118.65234375*b_wtsl_0_1_2_3 + 158.203125*b_wtsl_0_1_2_4 + 39.55078125*b_wtsl_0_2_0_1 + 79.1015625*b_wtsl_0_2_0_2 + 118.65234375*b_wtsl_0_2_0_3 + 158.203125*b_wtsl_0_2_0_4 + 39.55078125*b_wtsl_0_2_1_1 + 79.1015625*b_wtsl_0_2_1_2 + 118.65234375*b_wtsl_0_2_1_3 + 158.203125*b_wtsl_0_2_1_4 + 39.55078125*b_wtsl_0_2_2_1 + 79.1015625*b_wtsl_0_2_

In [14]:


solver = plp.GUROBI_CMD()
prob.solve(solver=solver)

Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/f5b54b2dc69a4603ae9d6995bec068e4-pulp.lp
Reading time = 0.02 seconds
Total_cost: 9280 rows, 5292 columns, 40352 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9280 rows, 5292 columns and 40352 nonzeros
Model fingerprint: 0xef82b41a
Variable types: 0 continuous, 5292 integer (5292 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6300 rows and 3269 columns
Presolve time: 0.07s
Presolved: 2980 rows, 2023 columns, 12188 nonzeros
Variable types: 0 continuous, 2023 integer (2023 binary)

R

1

In [15]:
prob.status

1

In [32]:
#TODO - add secondary objective that tasks should be at the same station for the same model
def generate_task_assignments_df_stochastic(pulp_prob, sequences, instances):
    '''Shows task assignments for fixed and model dependent task assignment'''
    task_assignments = []
    labor_assignments = []
    for v in pulp_prob.variables():
        print(v.name, v.varValue)
        if round(v.varValue) > 0:
            
            if 'x_wsoj' in v.name:
                sequence = int(v.name.split('_')[2])
                item = int(v.name.split('_')[5])
                model = sequences[sequence]['sequence'][item]
                #change the task number to match with the instances
                task = str(int(v.name.split('_')[4])+1)
                task_time = instances[model]['task_times'][task]
                assignment = {'scenario':v.name.split('_')[2], 'station': v.name.split('_')[3],'sequence_loc':item, 'model':model  , 'task': task, 'task_times': task_time}
                task_assignments.append(assignment)
            elif 'b_wtsl' in v.name:
                model = sequences[int(v.name.split('_')[2])]['sequence'][int(v.name.split('_')[4])]
                labor = {'scenario':v.name.split('_')[2], 'stage':v.name.split('_')[3], 'station': v.name.split('_')[4], 'model':model, 'workers': int(v.name.split('_')[5]) }
                labor_assignments.append(labor)

    #turns task_assignments into a dataframe
    task_assignments_df = pd.DataFrame(task_assignments)
    labor_assignments_df = pd.DataFrame(labor_assignments)
    #concatenates the 'task' column in task_assignments_df if the 'station' and 'model' columns are the same
    task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
    labor_assignments_df['sequence_loc'] = labor_assignments_df['stage'].astype(int) - labor_assignments_df['station'].astype(int)
    labor_assignments_df = labor_assignments_df[labor_assignments_df['sequence_loc'] >= 0]
    #concatenates the 'task' column in labor_assignments_df if the 'station' and 'model' columns are the same
    # labor_assignments_df = labor_assignments_df.groupby(['model','station'])[ 'workers'].apply( max).reset_index()
    #print(labor_assignments_df)
    #concatenates the 'workers' column in labor_assignments_df  to the task_assignments_df if the 'station' and 'model' columns are the same
    #task_assignments_df = pd.merge(task_assignments_df, labor_assignments_df, on=['station', 'model'])
    return task_assignments_df, labor_assignments_df
task_assignment, labor_assignment = generate_task_assignments_df_stochastic(prob, final_sequences, test_instances)
task_assignment

b_wtsl_0_0_0_0 0.0
b_wtsl_0_0_0_1 0.0
b_wtsl_0_0_0_2 0.0
b_wtsl_0_0_0_3 0.0
b_wtsl_0_0_0_4 1.0
b_wtsl_0_0_1_0 1.0
b_wtsl_0_0_1_1 0.0
b_wtsl_0_0_1_2 0.0
b_wtsl_0_0_1_3 0.0
b_wtsl_0_0_1_4 0.0
b_wtsl_0_0_2_0 1.0
b_wtsl_0_0_2_1 0.0
b_wtsl_0_0_2_2 0.0
b_wtsl_0_0_2_3 0.0
b_wtsl_0_0_2_4 0.0
b_wtsl_0_1_0_0 0.0
b_wtsl_0_1_0_1 0.0
b_wtsl_0_1_0_2 0.0
b_wtsl_0_1_0_3 0.0
b_wtsl_0_1_0_4 1.0
b_wtsl_0_1_1_0 0.0
b_wtsl_0_1_1_1 0.0
b_wtsl_0_1_1_2 1.0
b_wtsl_0_1_1_3 0.0
b_wtsl_0_1_1_4 0.0
b_wtsl_0_1_2_0 1.0
b_wtsl_0_1_2_1 0.0
b_wtsl_0_1_2_2 0.0
b_wtsl_0_1_2_3 0.0
b_wtsl_0_1_2_4 0.0
b_wtsl_0_2_0_0 0.0
b_wtsl_0_2_0_1 0.0
b_wtsl_0_2_0_2 0.0
b_wtsl_0_2_0_3 0.0
b_wtsl_0_2_0_4 1.0
b_wtsl_0_2_1_0 0.0
b_wtsl_0_2_1_1 0.0
b_wtsl_0_2_1_2 0.0
b_wtsl_0_2_1_3 1.0
b_wtsl_0_2_1_4 0.0
b_wtsl_0_2_2_0 0.0
b_wtsl_0_2_2_1 0.0
b_wtsl_0_2_2_2 0.0
b_wtsl_0_2_2_3 0.0
b_wtsl_0_2_2_4 1.0
b_wtsl_0_3_0_0 0.0
b_wtsl_0_3_0_1 0.0
b_wtsl_0_3_0_2 1.0
b_wtsl_0_3_0_3 0.0
b_wtsl_0_3_0_4 0.0
b_wtsl_0_3_1_0 0.0
b_wtsl_0_3_1_1 0.0
b_wtsl_0_3_1

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_26759/1872767633.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


,scenario,station,sequence_loc,model,task,task_times
0,0,0,0,A,"1,2,4,6,7,8,10",1176
1,0,0,1,A,"1,3,4,5,6,9,10",1200
2,0,0,2,A,"1,12,2,4,6,7,8",1133
3,0,0,3,A,"1,11,2,5,7,9",573
4,0,1,0,A,"11,13,17,3,5",538
...,...,...,...,...,...,...
187,9,1,3,B,"11,12,14,3,5,7,10",1192
188,9,2,0,B,"13,14,15,16,17,18,19,20,9",1143
189,9,2,1,A,"14,16,17,18,19,20,5,9",1161
190,9,2,2,A,"13,16,17,18,20,5,9,10",1191


In [24]:
labor_assignment.head(20)

,scenario,stage,station,model,workers,sequence_loc
0,0,0,0,A,4,0
3,0,1,0,A,4,1
4,0,1,1,A,2,0
6,0,2,0,A,4,2
7,0,2,1,A,3,1
8,0,2,2,A,4,0
9,0,3,0,A,2,3
10,0,3,1,A,2,2
11,0,3,2,A,3,1
12,0,4,0,A,0,4


In [35]:
task_seq = task_assignment[['scenario','station', 'task','task_times', 'sequence_loc']]
#merging labor and task sequence dataframes
labor_task_seq = pd.merge(labor_assignment, task_seq, on=['scenario','station', 'sequence_loc'], how='left')

In [36]:
labor_task_seq.head(29)


,scenario,stage,station,model,workers,sequence_loc,task,task_times
0,0,0,0,A,4,0,"1,2,4,6,7,8,10",1176.0
1,0,1,0,A,4,1,"1,3,4,5,6,9,10",1200.0
2,0,1,1,A,2,0,"11,13,17,3,5",538.0
3,0,2,0,A,4,2,"1,12,2,4,6,7,8",1133.0
4,0,2,1,A,3,1,"11,13,17,18,2,7,8",874.0
5,0,2,2,A,4,0,"12,14,15,16,18,19,20,9",1168.0
6,0,3,0,A,2,3,"1,11,2,5,7,9",573.0
7,0,3,1,A,2,2,"3,5,9,10",565.0
8,0,3,2,A,3,1,"12,14,15,16,19,20",808.0
9,0,4,0,A,0,4,NaN,NaN


In [ ]:
task_assignment.to_csv('task_assignment.csv', sep=' ')
labor_assignment.to_csv('labor_assignment.csv', sep=' ')